In [61]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd

In [62]:
## Load the ANN trained model,scaler pickle, one hot encoder pickle
model = load_model("model.h5")

## load the scaler and one hot encoder
with open('geo_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)


In [68]:
## Example Input data 
input_data = {
    'CreditScore': 600,   # <-- fix spelling
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 100000
}


In [69]:
## one hot encoding geography
geo_encoded_input = label_encoder.transform([[input_data['Geography']]]).toarray()
geo_ecoded_df = pd.DataFrame(geo_encoded_input, columns=label_encoder.get_feature_names_out(['Geography']))
geo_ecoded_df


/Users/adityagole/Desktop/Ai-tutorials/streamlit/venv/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [70]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,100000


In [71]:
## Encode categorical variable 
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,100000


In [72]:
##Concatenate one hot encoded 

input_df = pd.concat([input_df.drop("Geography", axis=1), geo_ecoded_df], axis=1)
input_df    

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,100000,1.0,0.0,0.0


In [73]:
## scaling the input data 
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.00749869,  1.00150113,
        -0.57946723, -0.57638802]])

In [74]:
##predict the churn
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 37ms/step


array([[0.02269175]], dtype=float32)

In [75]:
prediction_proba = prediction[0][0]

In [76]:
prediction_proba

0.022691755

In [77]:
if prediction_proba > 0.5:
    print("Prediction is likely to churn")
else:
    print("Prediction is not likely to churn")

Prediction is not likely to churn
